## Implementing QUBO to solve the Knapsack Problem

In [45]:
import numpy as np

In [46]:
items_values = {"⚽️": 8, "💻": 47, "📸": 10, "📚": 5, "🎸": 16}
values_list = [8, 47, 10, 5, 16]

In [47]:
items_weight = {"⚽️": 3, "💻": 11, "📸": 14, "📚": 19, "🎸": 5}
weights_list = [3, 11, 14, 19, 5]

In [48]:
maximum_weight = 26

In [49]:
def sum_weight(bitstring, items_weight):
    weight = 0
    for n, i in enumerate(items_weight):
        if bitstring[n] == "1":
            weight += i
    return weight


def sum_values(bitstring, items_value):
    value = 0
    for n, i in enumerate(items_value):
        if bitstring[n] == "1":
            value += i
    return value

In [50]:
items = list(items_values.keys())
n_items = len(items)
combinations = {}
max_value = 0
for case_i in range(2**n_items):
    combinations[case_i] = {}
    bitstring = np.binary_repr(
        case_i, n_items
    )
    combinations[case_i]["items"] = [items[n] for n, i in enumerate(bitstring) if i == "1"]
    combinations[case_i]["value"] = sum_values(bitstring, values_list)
    combinations[case_i]["weight"] = sum_values(bitstring, weights_list)
    if (
        combinations[case_i]["value"] > max_value
        and combinations[case_i]["weight"] <= maximum_weight
    ):
        max_value = combinations[case_i]["value"]
        optimal_solution = {
            "items": combinations[case_i]["items"],
            "value": combinations[case_i]["value"],
            "weight": combinations[case_i]["weight"],
        }

print(
    f"The best combination is {optimal_solution['items']} with a total value: {optimal_solution['value']} and total weight {optimal_solution['weight']} "
)

The best combination is ['⚽️', '💻', '🎸'] with a total value: 71 and total weight 19 


## QUBO
minimize x<sup>T</sup>Qx

* x -> vector of binary variables
* Q -> matrix of linear/quadratic terms

In [51]:
Q = -np.diag(list(items_values.values()))
x_opt = np.array(
    [[1 if i in optimal_solution["items"] else 0] for i in items_values.keys()]
)
opt_str = "".join(str(i[0]) for i in x_opt)
min_cost = (x_opt.T @ Q @ x_opt)[0, 0]

In [63]:
N = round(np.ceil(np.log2(maximum_weight + 1)))
print(N)
weights = list(items_weight.values()) + [2**k for k in range(N)]
# print(weights)
QT = np.pad(Q, ((0, N), (0, N)))
# print(QT)
n_qubits = len(QT)
lambd = 100
for i in range(len(QT)):
    QT[i, i] += lambd * weights[i] * (weights[i] - 2 * maximum_weight)
    for j in range(i + 1, len(QT)):
        QT[i, j] += 2 * lambd * weights[i] * weights[j]

print(QT)

5
[[-14708   6600   8400  11400   3000    600   1200   2400   4800   9600]
 [     0 -45147  30800  41800  11000   2200   4400   8800  17600  35200]
 [     0      0 -53210  53200  14000   2800   5600  11200  22400  44800]
 [     0      0      0 -62705  19000   3800   7600  15200  30400  60800]
 [     0      0      0      0 -23516   1000   2000   4000   8000  16000]
 [     0      0      0      0      0  -5100    400    800   1600   3200]
 [     0      0      0      0      0      0 -10000   1600   3200   6400]
 [     0      0      0      0      0      0      0 -19200   6400  12800]
 [     0      0      0      0      0      0      0      0 -35200  25600]
 [     0      0      0      0      0      0      0      0      0 -57600]]


In [64]:
offset = lambd * maximum_weight**2
print(offset)

67600


In [65]:
optimal = np.binary_repr(maximum_weight - optimal_solution["weight"], N)[::-1]
print(optimal)
x_opt_slack = np.concatenate(
    (x_opt, np.array([[int(i)] for i in optimal]))
)  
opt_str_slack = "".join(str(i[0]) for i in x_opt_slack)
cost = (x_opt_slack.T @ QT @ x_opt_slack)[0, 0] + offset  
print(f"Cost:{cost}")

11100
Cost:-71
